In [4]:
import datetime as dt
from jinja2 import Template
from pytz import timezone
from matplotlib import pyplot as plt
import pandas as pd
from scipy.optimize import linprog
from IPython.display import HTML
from money_squirrel.bin import utils

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 7)

COINS = utils.load_config("coins")

def display_all(df):
    return HTML(df.to_html())

2017-07-07 07:33:14,084|money_squirrel.bin.utils|INFO|Instantiated logger w/ name=money_squirrel.bin.utils
2017-07-07 07:33:14,087|money_squirrel.bin.utils|INFO|Loading JSON from /Users/ravdek/code/money_squirrel/money_squirrel/config/coins.json


# optimization

Given some period of time, we'd like to know what were the optimal buys and sells during that period. We can express this as a linear programming problem:
$$
\text{minimize}\quad c^{T}\ast x\quad :\quad A_{ub}\ast x \le b_{ub}
$$
where $c^{T}\ast x$ is proportional to -(balance after trades), $A_{ub}, b_{ub}$ describe our constrains, and $x$ describes our trades.

We want the coefficients of $x$ to describe the **percent of our balance** of product spent on a buy/sell with
```
x.index = [('BTC-USD', 'buy'), ..., ('LTC-USD', 'sell')]
```
To simplify, we can make everything a buy by reversing the "sell" pairs, and always let the coefficients of $x$ refer to the percent of the currency being withdrawn during a purchase:
```
('LTC-USD', 'sell') ~ ('USD', 'LTC')
x.index = [('BTC', 'USD'), ('USD', 'BTC'), ..., ('USD', 'LTC')]
```

We'll also include pairs for holding: `('BTC-BTC')`, etc.

We'll assume that starting balance, high/low prices during the time period, and closing prices are known. (We'll have them known during training and predicted in real time). Then our constraints become:
$$
0 \le x_{(i, j)} \le 1\quad \forall (i, j)
\sum_i x_{(i, j)} = 1 \forall j
$$

We want to get a nice expression for the vector $c$, so we'll do some calculations. Let $b_i$ denote the starting balances of each currency and $B_i$ the ending balances during the time period. Let $p_i$ be our close price for each product in $USD$ and let $\hat{p}_{i, j}$ be the price of currency $i$ WRT currency $j$ at which we will make our purchased. Let $\alpha_{i, j}$ denote the fee for each trade pair. Note if we spend $x$ on a trade, we only use $(1-\alpha_{i, j})\ast x$ towards our purchase. For holding a product, $\alpha_{i, i} = 0$

$$
B_i = - (\text{amount withdrawn during trades}) + (\text{amount deposited during trades}) = - \sum_j b_i \ast x_{j, i} + \sum_j \frac{(1-\alpha_{i,j})\ast b_j}{\hat{p}_{i,j}}x_{i,j}
$$

If we cash out (and include cash-out costs), the final worth of our account in USD (which is what we'll want to optimize) is...

$$
finalWorth = \sum_i (1 - \alpha_{i, USD})\ast p_i \ast B_i
$$

Expanding as above
$$
= \sum_i (1 - \alpha_{i, USD})\ast p_i \ast [- \sum_j b_i \ast x_{j, i} + \sum_j \frac{(1-\alpha_{i,j})\ast b_j}{\hat{p}_{i,j}}x_{i,j}]
$$

Distributing terms
$$
= - \sum_{i,j} (1 - \alpha_{i,USD}) \ast p_i \ast b_i \ast x_{j,i}
+ \sum_{i,j} (1 - \alpha_{i,USD}) \ast p_i \frac{(1-\alpha_{i,j})\ast b_j}{\hat{p}_{i,j}} x_{i,j}
$$

Flipping i and j indicies in the withdraw terms
$$
= - \sum_{i,j} (1 - \alpha_{j,USD}) \ast p_j \ast b_j \ast x_{i,j}
+ \sum_{i,j} (1 - \alpha_{i,USD}) \ast p_i \frac{(1-\alpha_{i,j})\ast b_j}{\hat{p}_{i,j}} x_{i,j}
$$

Smushing together
$$
= \sum_{i,j} [(\alpha_{j,USD} - 1) \ast p_j \ast b_j + (1 - \alpha_{i,USD}) \ast p_i \frac{(1-\alpha_{i,j})\ast b_j}{\hat{p}_{i,j}}] x_{i,j}
$$

So that when we throw a minus sign on
$$
c_{i,j} = b_j \ast [(1 - \alpha_{j,USD}) \ast p_j + (\alpha_{i,USD} - 1) \ast p_i \frac{(1-\alpha_{i,j})}{\hat{p}_{i,j}}]
$$

In [9]:
FEES = {
    'BTC': .0025,
    'LTC': .0025,
    'ETH': .003,
    'USD': 0.0
}


class MarketBuyOptimizer(object):
    """
    We assume that the purchase prices are the same as the start prices
    as one would assume to be the case for a market order.
    """
    
    def __init__(self, start_balance, start_prices, end_prices):
        self.start_balance = start_balance
        self.start_prices = start_prices
        self.end_prices = end_prices
        # We want pairs to be a list so that 
        # we can use the indicies for building arrays
        self.pairs = []
        for k in start_prices.keys():
            i, j = k.split('-')
            self.pairs.append((i, j))
            self.pairs.append((j, i))
        for currency in {p[0] for p in self.pairs}:
            self.pairs.append((currency, currency))

        
    def _fee(self, i, j):
        if i == j:
            return 0
        return max([FEES[i], FEES[j]])
    
    def _final_price_usd(self, i):
        if i == 'USD':
            return 1
        else:
            return self.end_prices['-'.join([i, 'USD'])]
        
    def _purchase_price(self, i, j):
        # Here is that market price assumption
        if i == j:
            return 1
        elif '-'.join([i, j]) in self.start_prices.keys():
            return self.start_prices['-'.join([i, j])]
        else:
            return 1.0 / self.start_prices['-'.join([j, i])]
        
    def _cost_coef(self, i, j):
        
        cost_before_balance = (
            (1.0 - self._fee(j, 'USD')) 
            * self._final_price_usd(j)
        ) + (
            (self._fee(i, 'USD') - 1.0) 
            * self._final_price_usd(i)
            * (1 - self._fee(i, j))
            / self._purchase_price(i, j)
        )
        return self.start_balance[j] * cost_before_balance
    
    def _cost_arr(self):
        return pd.np.array([
            self._cost_coef(*pair)
            for pair in self.pairs
        ])
    
    def _bounds(self):
        a_ub = []
        b_ub = []
        a_eq = []
        b_eq = []
        
        for i in range(len(self.pairs)):
            # every coef must be >= 0
            a_ub.append([-1 if j==i else 0 for j in range(len(self.pairs))])
            b_ub.append(0)
            # every coef must be <= 1
            a_ub.append([1 if j==i else 0 for j in range(len(self.pairs))])
            b_ub.append(1)
            
        # Together with holds (which is a self sell in our syntax)
        # we must sell the entireity of a product
        for currency in set([p[1] for p in self.pairs]):
            a_eq.append([1 if p[1]==currency else 0 for p in self.pairs])
            b_eq.append(1)
            
        return pd.np.array(a_ub), pd.np.array(b_ub), pd.np.array(a_eq), pd.np.array(b_eq)
    
    def fit(self):
        
        c = self._cost_arr()
        a_ub, b_ub, a_eq, b_eq = self._bounds()
        self.results = linprog(c, a_ub, b_ub, a_eq, b_eq)

# Lets run some simulations

In [10]:
# BTC dive bombs
# ETH hockey sticks
# LTC does nothing

start_balance = {
    'USD': 50,
    'BTC': 50,
    'ETH': 50,
    'LTC': 50
}
start_prices = {
    'BTC-USD': 1000,
    'ETH-USD': 5,
    'LTC-USD': 2
}
end_prices = {
    'BTC-USD': 500,
    'ETH-USD': 20,
    'LTC-USD': 2.2
}
mbo = MarketBuyOptimizer(
    start_balance=start_balance,
    start_prices=start_prices,
    end_prices=end_prices
)
mbo.fit()
zip(mbo.pairs, mbo.results.x)

[(('ETH', 'USD'), 1.0),
 (('USD', 'ETH'), 0.0),
 (('LTC', 'USD'), 0.0),
 (('USD', 'LTC'), 0.0),
 (('BTC', 'USD'), 0.0),
 (('USD', 'BTC'), 1.0),
 (('LTC', 'LTC'), 1.0),
 (('ETH', 'ETH'), 1.0),
 (('USD', 'USD'), 0.0),
 (('BTC', 'BTC'), 0.0)]

In [13]:
# Everything crashes

start_balance = {
    'USD': 50,
    'BTC': 50,
    'ETH': 50,
    'LTC': 50
}
start_prices = {
    'BTC-USD': 1000,
    'ETH-USD': 5,
    'LTC-USD': 2
}
end_prices = {
    'BTC-USD': 1,
    'ETH-USD': 1,
    'LTC-USD': 1
}
mbo = MarketBuyOptimizer(
    start_balance=start_balance,
    start_prices=start_prices,
    end_prices=end_prices
)
mbo.fit()
zip(mbo.pairs, mbo.results.x)

[(('ETH', 'USD'), 0.0),
 (('USD', 'ETH'), 1.0),
 (('LTC', 'USD'), 0.0),
 (('USD', 'LTC'), 1.0),
 (('BTC', 'USD'), 0.0),
 (('USD', 'BTC'), 1.0),
 (('LTC', 'LTC'), 0.0),
 (('ETH', 'ETH'), 0.0),
 (('USD', 'USD'), 1.0),
 (('BTC', 'BTC'), 0.0)]

In [11]:
# BTC doubles
# ETH doubles
# LTC raises a little

start_balance = {
    'USD': 50,
    'BTC': 50,
    'ETH': 50,
    'LTC': 50
}
start_prices = {
    'BTC-USD': 1000,
    'ETH-USD': 5,
    'LTC-USD': 2
}
end_prices = {
    'BTC-USD': 2000,
    'ETH-USD': 10,
    'LTC-USD': 2.5
}
mbo = MarketBuyOptimizer(
    start_balance=start_balance,
    start_prices=start_prices,
    end_prices=end_prices
)
mbo.fit()
zip(mbo.pairs, mbo.results.x)

[(('ETH', 'USD'), 0.0),
 (('USD', 'ETH'), 0.0),
 (('LTC', 'USD'), 0.0),
 (('USD', 'LTC'), 0.0),
 (('BTC', 'USD'), 1.0),
 (('USD', 'BTC'), 0.0),
 (('LTC', 'LTC'), 1.0),
 (('ETH', 'ETH'), 1.0),
 (('USD', 'USD'), 0.0),
 (('BTC', 'BTC'), 1.0)]

In [12]:
# Same scenario except now we have LTC-BTC and ETH-BTC pairs

start_balance = {
    'USD': 50,
    'BTC': 50,
    'ETH': 50,
    'LTC': 50
}
start_prices = {
    'BTC-USD': 10,
    'ETH-USD': 10,
    'LTC-USD': 10,
    'ETH-BTC': 1,
    'LTC-BTC': 1
}
end_prices = {
    'BTC-USD': 100,
    'ETH-USD': 100,
    'LTC-USD': 20,
    'ETH-BTC': 1,
    'LTC-BTC': 5
}
mbo = MarketBuyOptimizer(
    start_balance=start_balance,
    start_prices=start_prices,
    end_prices=end_prices
)
mbo.fit()
sorted(zip(mbo.pairs, mbo.results.x))

[(('BTC', 'BTC'), 1.0),
 (('BTC', 'ETH'), 0.0),
 (('BTC', 'LTC'), 1.0),
 (('BTC', 'USD'), 1.0),
 (('ETH', 'BTC'), 0.0),
 (('ETH', 'ETH'), 1.0),
 (('ETH', 'USD'), 0.0),
 (('LTC', 'BTC'), 0.0),
 (('LTC', 'LTC'), 0.0),
 (('LTC', 'USD'), 0.0),
 (('USD', 'BTC'), 0.0),
 (('USD', 'ETH'), 0.0),
 (('USD', 'LTC'), 0.0),
 (('USD', 'USD'), 0.0)]